In [ ]:
import numpy as np
from channel import WirelessChannel
from network import WirelessNetwork
from regnn import REGNN, train_unconstrained, train_primal_dual
from utils import plot_pathloss, ribbon_by_distance, scatter_network, avg_loss_cap_pow

d0, gamma, s, N0 = 1.0, 2.2, 2.0, 1e-6

In [ ]:
wc = 50.0  # (not used here; just keeping constants nearby)
ch = WirelessChannel(d0=d0, gamma=gamma, s=s, N0=N0, rng=7)
dvals = np.linspace(1, 100, 100)
Eh = ch.pathloss(dvals)
plot_pathloss(dvals, Eh)


In [ ]:
samples = []
for d in dvals:
    samples.append(ch.sample_fading(q=100, d=d))
ribbon_by_distance(dvals, samples, ylabel='h', title='Fading range vs distance')

In [ ]:
P_const = 0.05
cap_samples = [ch.capacity(P_const, np.asarray(x)) for x in samples]
ribbon_by_distance(dvals, cap_samples, ylabel='log(1 + hp/N0)', title='Capacity range vs distance')

In [ ]:
wx, wy, wc = 200, 100, 50
rho = 0.05
n = int(rho * wx * wy)  
net = WirelessNetwork(n=n, wx=wx, wy=wy, wc=wc, d0=d0, gamma=gamma, s=s, N0=N0, rng=123)
scatter_network(net.tx, net.rx, wx, wy)

H_q = net.sample_H(q=3)

In [ ]:
wx, wy, wc = 80, 40, 20
n = int(0.05 * wx * wy)
net3 = WirelessNetwork(n=n, wx=wx, wy=wy, wc=wc, d0=d0, gamma=gamma, s=s, N0=N0, rng=42)

model = REGNN(n_features=[8,4,1], K=5)
model = train_unconstrained(model, net3, steps=200, batch_graphs=100, mu=0.01, lr=1e-2)

H_eval = net3.sample_H(q=160)
avg_loss, avg_caps_per_node, avg_pow_per_node = avg_loss_cap_pow(model, H_eval, N0)
avg_loss, avg_caps_per_node[:5], avg_pow_per_node[:5]

In [ ]:
p_budget = 1e-3
model_c = REGNN(n_features=[8,4,1], K=5)
model_c, mu_vec = train_primal_dual(model_c, net3, steps=200, batch_graphs=100,
                                    p_budget=p_budget, lr_primal=1e-2, lr_dual=1e-3)

H_eval = net3.sample_H(q=100)
avg_loss_c, avg_caps_node_c, avg_pow_node_c = avg_loss_cap_pow(model_c, H_eval, N0)
(avg_loss_c, avg_caps_node_c[:5], avg_pow_node_c[:5], mu_vec[:5])
